In [ ]:
import pandas as pd
import numpy as np
from google.colab import files

In [ ]:
old = pd.read_csv("tweet_emotions.csv")
old.shape

FileNotFoundError: ignored

In [ ]:
def sorter(emotion):
  val = emotion
  if emotion == 'empty':
    val = 'sadness'
  elif emotion == 'enthusiasm':
    val = 'joy'
  elif emotion == 'worry':
    val = 'fear'
  elif emotion == 'fun':
    val = 'joy'
  elif emotion == 'hate':
    val = 'anger'
  elif emotion == 'happiness':
    val = 'joy'
  elif emotion == 'boredom':
    val = 'neutral'
  elif emotion == 'relief':
    val = 'joy'
  return val

In [ ]:
old['relabeled'] = old['sentiment'].apply(sorter)

In [ ]:
old['relabeled'].unique()

array(['sadness', 'joy', 'neutral', 'fear', 'surprise', 'love', 'anger'],
      dtype=object)

In [ ]:
old.groupby('relabeled').agg("count")

,tweet_id,sentiment,content
relabeled,,,
anger,1433,1433,1433
fear,8459,8459,8459
joy,9270,9270,9270
love,3842,3842,3842
neutral,8817,8817,8817
sadness,5992,5992,5992
surprise,2187,2187,2187


In [ ]:
old = old.drop_duplicates('content', keep=False)

In [ ]:
old.groupby('relabeled').agg("count")

,tweet_id,sentiment,content
relabeled,,,
anger,1432,1432,1432
fear,8421,8421,8421
joy,9216,9216,9216
love,3770,3770,3770
neutral,8759,8759,8759
sadness,5963,5963,5963
surprise,2177,2177,2177


In [ ]:
to_mix = old[['content', 'relabeled']]
to_mix['emotions'] = to_mix['relabeled']
to_mix['text'] = to_mix['content']
to_mix = to_mix.drop(['content', 'relabeled'],axis=1)


<ipython-input-10-51227e45672d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_mix['emotions'] = to_mix['relabeled']


In [ ]:
to_mix.shape

(39738, 2)

In [ ]:
clean_train = pd.read_csv("train_data_imbalanced_stratified_no_dupe_34k_10k.csv")
clean_val = pd.read_csv("validation_data_imbalanced_stratified_no_dupe_34k_10k.csv")
print(clean_train.shape)
print(clean_val.shape)

(34164, 3)
(10250, 3)


In [ ]:
full_clean = pd.concat([clean_train, clean_val])
print(full_clean.shape)
full_clean = full_clean.drop_duplicates('text', keep=False)
full_clean = full_clean.drop(['Unnamed: 0'], axis=1)
print(full_clean.shape)

(44414, 3)
(44414, 2)


In [ ]:
full_clean.head()

,text,emotions
0,i am feeling really impatient,anger
1,i have a feeling that i am going to using thes...,joy
2,i feel like my life has just gotten considerab...,sadness
3,i had to sharpen up and concentrate and stop f...,joy
4,i just feel so strongly because of how loyal t...,love


In [ ]:
to_mix.head()

,emotions,text
0,sadness,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,joy,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [ ]:
mixed_data = pd.concat([full_clean, to_mix])
print(mixed_data.shape)

(84152, 2)


In [ ]:
mixed_data = mixed_data.drop_duplicates('text', keep=False)
mixed_data.shape

(82142, 2)

In [ ]:
from sklearn.model_selection import train_test_split

# a function to split data (doing this a lot in this notebook)
def data_splitter(data_df, test_pct, train_pct):
  return train_test_split(data_df['text'], data_df['emotions'],
                                                      test_size=test_pct,
                                                      train_size=train_pct,
                                                      stratify=data_df['emotions'],
                                                      shuffle=True,
                                                      random_state=11) # make sure this is reliable

In [ ]:
def check_overlap(text_of_df1, text_of_df2):
  # Check if there are any records in train that are in val
  overlap = text_of_df1.isin(text_of_df2)
  print(f"Number of overlapping records (df1 in df2): {overlap.sum()}")

  # Check if there are any records in val that are in train
  overlap = text_of_df2.isin(text_of_df1)
  print(f"Number of overlapping records (df2 in df1): {overlap.sum()}")

In [ ]:
X_train, X_val, y_train, y_val = data_splitter(mixed_data, test_pct=0.15, train_pct=0.25)

In [ ]:
check_overlap(X_train, X_val)

Number of overlapping records (df1 in df2): 0
Number of overlapping records (df2 in df1): 0


In [ ]:
check_overlap(y_train, y_val)

Number of overlapping records (df1 in df2): 20535
Number of overlapping records (df2 in df1): 12322


In [ ]:
print(len(X_train))
print(len(X_val))

20535
12322


In [ ]:
def get_proportions(data):
  # get the value counts of each unique value
  counts = data.value_counts()

  # convert the counts to percentages
  percentages = counts / counts.sum() * 100

  return pd.DataFrame({'count': counts, 'percentage': percentages})  

In [ ]:
get_proportions(mixed_data['emotions'])

,count,percentage
joy,24329,29.618222
sadness,19520,23.763726
fear,13049,15.885905
neutral,7755,9.440968
anger,7546,9.186531
love,6617,8.055562
surprise,3326,4.049086


In [ ]:
get_proportions(y_train)

,count,percentage
joy,6082,29.617726
sadness,4880,23.764305
fear,3262,15.885074
neutral,1939,9.442415
anger,1886,9.184319
love,1654,8.054541
surprise,832,4.051619


In [ ]:
get_proportions(y_val)

,count,percentage
joy,3650,29.621815
sadness,2928,23.762376
fear,1957,15.882162
neutral,1163,9.438403
anger,1132,9.186820
love,993,8.058757
surprise,499,4.049667


In [ ]:
new_train_df = pd.DataFrame({'text': X_train, 'emotions': y_train})
new_val_df = pd.DataFrame({'text': X_val, 'emotions': y_val})


In [ ]:
new_train_df.to_csv('train_20k_12k_mixed_strat_clean_kaggle_230404.csv')
new_val_df.to_csv('validation_20k_12k_mixed_strat_clean_kaggle_230404.csv')
files.download('train_20k_12k_mixed_strat_clean_kaggle_230404.csv')
files.download('validation_20k_12k_mixed_strat_clean_kaggle_230404.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#now we create dataset for clean vs unclean classification task
to_mix['source'] = 'unclean'

In [ ]:
full_clean['source'] = 'clean'

In [ ]:
to_mix.head(10)

,emotions,text,source
0,sadness,@tiffanylue i know i was listenin to bad habi...,unclean
1,sadness,Layin n bed with a headache ughhhh...waitin o...,unclean
2,sadness,Funeral ceremony...gloomy friday...,unclean
3,joy,wants to hang out with friends SOON!,unclean
4,neutral,@dannycastillo We want to trade with someone w...,unclean
5,fear,Re-pinging @ghostridah14: why didn't you go to...,unclean
6,sadness,"I should be sleep, but im not! thinking about ...",unclean
7,fear,Hmmm. http://www.djhero.com/ is down,unclean
8,sadness,@charviray Charlene my love. I miss you,unclean
9,sadness,@kelcouch I'm sorry at least it's Friday?,unclean


In [ ]:
full_clean.head(10)

,text,emotions,source
0,i am feeling really impatient,anger,clean
1,i have a feeling that i am going to using thes...,joy,clean
2,i feel like my life has just gotten considerab...,sadness,clean
3,i had to sharpen up and concentrate and stop f...,joy,clean
4,i just feel so strongly because of how loyal t...,love,clean
5,i feel priviledged to get to spend time with t...,joy,clean
6,i feel as though i cannot have a casual writte...,joy,clean
7,i feel idiotic about it but i cant,sadness,clean
8,i dont expect anything better from takano but ...,sadness,clean
9,i feel super when i eat it,joy,clean


In [ ]:
to_mix_dist = to_mix[['text', 'source']]
clean_dist = full_clean[['text', 'source']]

In [ ]:
dist_mix = pd.concat([to_mix_dist, clean_dist])
dist_mix = dist_mix.drop_duplicates('text', keep=False)

In [ ]:
dist_mix.shape

(82142, 2)

In [ ]:
def data_splitter_class(data_df, test_pct, train_pct):
  return train_test_split(data_df['text'], data_df['source'],
                                                      test_size=test_pct,
                                                      train_size=train_pct,
                                                      stratify=data_df['source'],
                                                      shuffle=True,
                                                      random_state=11) # make sure this is reliable

In [ ]:
X_train, X_val, y_train, y_val = data_splitter_class(dist_mix, test_pct=0.15, train_pct=0.25)

In [ ]:
check_overlap(X_train, X_val)

Number of overlapping records (df1 in df2): 0
Number of overlapping records (df2 in df1): 0


In [ ]:
X_train.shape

(20535,)

In [ ]:
X_val.shape

(12322,)

In [ ]:
get_proportions(y_train)

,count,percentage
clean,10852,52.84636
unclean,9683,47.15364


In [ ]:
get_proportions(y_val)

,count,percentage
clean,6512,52.848564
unclean,5810,47.151436


In [ ]:
class_train_df = pd.DataFrame({'text': X_train, 'source': y_train})
class_val_df = pd.DataFrame({'text': X_val, 'source': y_val})

In [ ]:
class_train_df.to_csv('train_source_20k_12k_mixed_230404.csv')
class_val_df.to_csv('validation_source_20k_12k_mixed_230404.csv')
files.download('train_source_20k_12k_mixed_230404.csv')
files.download('validation_source_20k_12k_mixed_230404.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>